# Vector Index

HNSW index

## Payload index

+ keyword
+ integer
+ float
+ bool
+ geo
+ text

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
client = QdrantClient('localhost', port=6333)
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='hello_world_collection'), CollectionDescription(name='sparse_vector_collection'), CollectionDescription(name='multi_vector_collection'), CollectionDescription(name='star_charts'), CollectionDescription(name='wolt_food')])

In [2]:
# Create index
client.create_payload_index(
    collection_name='wolt_food',
    field_name='description',
    field_schema='keyword',
)

ResponseHandlingException: timed out

In [3]:
# Create index on nested fields
client.create_payload_index(
    collection_name='wolt_food',
    field_name='cafe.rating',
    field_schema='float',
)

ResponseHandlingException: timed out

In [4]:
# Full text index

client.create_payload_index(
    collection_name='wolt_food',
    field_name='description',
    field_schema=models.TextIndexParams(
        type='text',
        tokenizer=models.TokenizerType.WORD,
        min_token_len=3, 
        max_token_len=8,
        lowercase=True,
    )
)

ResponseHandlingException: timed out

Hierarchial Navigable Small World graph


In [6]:
# HNSW

client.create_collection(
    collection_name='hnsw_example',
    vectors_config=models.VectorParams(size=128, distance=models.Distance.COSINE),
    hnsw_config=models.HnswConfig(m=16, ef_construct=128)
)

ValidationError: 1 validation error for HnswConfig
full_scan_threshold
  Field required [type=missing, input_value={'m': 16, 'ef_construct': 128}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

## Scalar quantization(标量量化)

标量量化通过将高精度浮点向量（如float32）转换为低精度整数（如int8），实现32倍内存压缩，同时保留90%以上的搜索精度

In [7]:
client.create_collection(
    collection_name='scalar_quantization_collection',
    vectors_config=models.VectorParams(size=1024, distance=models.Distance.EUCLID),
    quantization_config=models.ScalarQuantization(
        scalar=models.ScalarQuantizationConfig(
            type=models.ScalarType.INT8,
            quantile=0.99,
            always_ram=True,
        ),
    ),
)

True

## Binary quantization

In [9]:
# Binary quantization

client.create_collection(
    collection_name='binary_quantization_collection',
    vectors_config=models.VectorParams(size=1024, distance=models.Distance.EUCLID),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(
            always_ram=True,
        ),
    ),
)

True

## Product quantization

In [10]:
# Product quantization

client.create_collection(
    collection_name='product_quantization_collection',
    vectors_config=models.VectorParams(size=1024, distance=models.Distance.COSINE),
    quantization_config=models.ProductQuantization(
        product=models.ProductQuantizationConfig(
            always_ram=True,
            compression=models.CompressionRatio.X16,
        ),
    ),
)

True

In [12]:
# Batch insert - product quantization
from random import uniform

insert_vectors = [[uniform(-0.999, 0.999) for vec in range(1024)] for x in range(1000)]
client.upsert(
    collection_name='product_quantization_collection',
    points=models.Batch(
        ids=[x for x in range(1000)],
         vectors=insert_vectors)
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
client.search(
    collection_name='product_quantization_collection',
    query_vector=[uniform(-0.999,0.999) for vec in range(1024)],
    search_params=models.SearchParams(
        quantization=models.QuantizationSearchParams(
            ignore=False, 
            rescore=True,
            oversampling=1.0,
        ),
    ),
)

/var/folders/z3/8v7jzndj50s8m09njkvjg18w0000gn/T/ipykernel_5531/1064012322.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  client.search(


[ScoredPoint(id=184, version=0, score=0.09928494, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=800, version=0, score=0.09822602, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=908, version=0, score=0.092406854, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=512, version=0, score=0.08154284, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=891, version=0, score=0.08088516, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=146, version=0, score=0.080251776, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=858, version=0, score=0.08019419, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=482, version=0, score=0.07955009, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=544, version=0, score=0.07819645, payload={}, vector=None, shard_key=None, order_value=None),
 ScoredP